## Convolutional Neural Network

reference: [https://wegonnamakeit.tistory.com/48](https://wegonnamakeit.tistory.com/48), [http://taewan.kim/post/cnn/](http://taewan.kim/post/cnn/)

### Basic CNN

![img](http://deeplearning.net/software/theano/_images/numerical_padding_strides.gif)

- Convolution: 이미지 위에 stride 값 만큼 filter(or kernel)을 이동시키면서 겹쳐지는 부분의 각 원소의 값을 모두 곱한 뒤 합산한 값을 출력하는 연산
- filter(kernel): output_channels(filter_channels) x input_channels x kernel_size x kernel_size
- Stride: filter를 sliding window 방식으로 한 번에 이동시키는 간격
- Padding: pad 크기 만큼 이미지의 상하좌우에 '0'으로 값을 채우는 것. output의 width, height 크기를 조절하기 위해 사용합니다. 
- input(image or features): Batch x Channel x Height x Width (Pytorch: BCHW format, Tensorflow: BHWC format)
  - Batch는 입력데이터의 묶음을 의미합니다. 입력데이터를 이미지 한 장으로 구성한다면 1 x C x H x W와 같습니다. 이미지의 크기가 256 x 256이고, RGB channel이라면 입력데이터는 1x3x256x256 입니다.
- output(features or feature map): Batch x filter_channels x computed_height x computed_width
  - computed_width = ((width - kernel_size + 2*pad) / stride) + 1
  - computed_height = ((height - kernel_size + 2*pad) / stride) + 1

In [ ]:
import torch
import torch.nn as nn

in_channels = 3
out_channels = 2
kernel_size = 3
stride = 2
pad = 1

# Basic 2D convolution
conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=pad)

print('conv.weight:\n', conv.weight) 
print('conv.weight.shape:\n', conv.weight.shape) # filter(kernel) is weights
print()
print('conv.bias:\n', conv.bias)
print('conv.bias.shape:\n', conv.bias.shape) # filter도 역시 bias를 가질 수 있습니다.
print()

input_data = torch.ones((1, 3, 256, 256)) # 256x256x3
print('input_data:\n', input_data)

output_features = conv(input_data)
print('output_features:\n', output_features)
print('output_features.shape:\n', output_features.shape)


마지막으로 torchvision 패키지가 제공하는 MNIST 이미지 데이터셋을 사용해서 

In [ ]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100 # 입력 데이터 묶음
learning_rate = 0.001

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

model = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))
